# LangGraph Section 5 Notes - Advanced RAG Flows

* Be extremely careful with version dependencies, as LangGraph and LangChain are in active development, and very sensitive to versions

# Step 1 - Ingestion Process

In [ ]:
# Solution will use vector store and ChromaDB (since it's local, fast and non persistent)